# Cognitive Analysis Dashboard

V5 Metacognitive Architecture Analysis Tool

This notebook provides visualization and analysis of cognitive events captured during agent execution.

## Features
- Thinking compliance rate by phase
- Most common thinking scenarios triggered
- Temptation encounter/resistance rates
- Decision point timeline visualization
- Reasoning coherence samples for manual review

In [ ]:
# Setup and imports
import sys
from pathlib import Path

# Add src to path for imports
sys.path.insert(0, str(Path.cwd().parent / "src"))

from compymac.trace_store import TraceStore, create_trace_store, CognitiveEvent
from compymac.temptations import Temptation, TEMPTATION_CATALOG
from compymac.swe_workflow import SWEPhase

from collections import Counter
from datetime import datetime
import json

In [ ]:
# Configuration
DB_PATH = None  # Use default path, or specify custom path
TRACE_ID = None  # Set to specific trace_id or None to analyze all

# Create trace store connection
store = create_trace_store(db_path=DB_PATH)
print(f"Connected to trace store")

## 1. Load Cognitive Events

In [ ]:
def load_cognitive_events(store: TraceStore, trace_id: str | None = None) -> list[CognitiveEvent]:
    """Load cognitive events from trace store."""
    if trace_id:
        return store.get_cognitive_events(trace_id)
    else:
        print("Please specify a TRACE_ID to analyze")
        return []

events = load_cognitive_events(store, TRACE_ID)
print(f"Loaded {len(events)} cognitive events")

## 2. Thinking Compliance Analysis

In [ ]:
def analyze_thinking_compliance(events: list[CognitiveEvent]) -> dict:
    """Analyze thinking compliance by phase and scenario."""
    thinking_events = [e for e in events if e.event_type == "think"]
    
    by_phase = {}
    for event in thinking_events:
        phase = event.phase or "unknown"
        if phase not in by_phase:
            by_phase[phase] = []
        by_phase[phase].append(event)
    
    scenario_counts = Counter()
    for event in thinking_events:
        trigger = event.metadata.get("trigger", "unspecified") if event.metadata else "unspecified"
        scenario_counts[trigger] += 1
    
    required_scenarios = [
        "before_claiming_completion",
        "before_advancing_to_fix",
        "before_git_operations",
    ]
    
    scenario_compliance = {}
    for scenario in required_scenarios:
        satisfied = any(
            scenario in (e.metadata.get("trigger", "") or "")
            for e in thinking_events
            if e.metadata
        )
        scenario_compliance[scenario] = satisfied
    
    return {
        "total_thinking_events": len(thinking_events),
        "by_phase": {k: len(v) for k, v in by_phase.items()},
        "scenario_counts": dict(scenario_counts),
        "required_scenario_compliance": scenario_compliance,
        "compliance_rate": sum(scenario_compliance.values()) / len(required_scenarios) if required_scenarios else 1.0,
    }

thinking_analysis = analyze_thinking_compliance(events)
print("=== Thinking Compliance Analysis ===")
print(f"Total thinking events: {thinking_analysis['total_thinking_events']}")
print(f"By phase: {thinking_analysis['by_phase']}")
print(f"Scenario triggers: {thinking_analysis['scenario_counts']}")
print(f"Required scenario compliance: {thinking_analysis['required_scenario_compliance']}")
print(f"Overall compliance rate: {thinking_analysis['compliance_rate']:.1%}")

## 3. Temptation Awareness Analysis

In [ ]:
def analyze_temptations(events: list[CognitiveEvent]) -> dict:
    """Analyze temptation encounter and resistance rates."""
    temptation_events = [e for e in events if e.event_type == "temptation_awareness"]
    
    by_type = Counter()
    recognized_count = 0
    resisted_count = 0
    
    for event in temptation_events:
        if event.metadata:
            temptation_type = event.metadata.get("temptation", "unknown")
            by_type[temptation_type] += 1
            if event.metadata.get("recognized", False):
                recognized_count += 1
            if event.metadata.get("resisted", False):
                resisted_count += 1
    
    total = len(temptation_events)
    return {
        "total_encountered": total,
        "by_type": dict(by_type),
        "recognized": recognized_count,
        "resisted": resisted_count,
        "recognition_rate": recognized_count / total if total > 0 else 1.0,
        "resistance_rate": resisted_count / total if total > 0 else 1.0,
    }

temptation_analysis = analyze_temptations(events)
print("=== Temptation Awareness Analysis ===")
print(f"Total encountered: {temptation_analysis['total_encountered']}")
print(f"By type: {temptation_analysis['by_type']}")
print(f"Recognition rate: {temptation_analysis['recognition_rate']:.1%}")
print(f"Resistance rate: {temptation_analysis['resistance_rate']:.1%}")

## 4. Decision Point Timeline

In [ ]:
def create_timeline(events: list[CognitiveEvent]) -> list[dict]:
    """Create a timeline of cognitive events."""
    timeline = []
    for event in sorted(events, key=lambda e: e.timestamp):
        timeline.append({
            "timestamp": datetime.fromtimestamp(event.timestamp).strftime("%H:%M:%S"),
            "type": event.event_type,
            "phase": event.phase or "unknown",
            "content_preview": (event.content[:100] + "...") if event.content and len(event.content) > 100 else event.content,
        })
    return timeline

timeline = create_timeline(events)
print("=== Cognitive Event Timeline ===")
for entry in timeline[:20]:
    print(f"[{entry['timestamp']}] {entry['type']:20} | {entry['phase']:15} | {entry['content_preview'] or '(no content)'}")
if len(timeline) > 20:
    print(f"... and {len(timeline) - 20} more events")

## 5. Reasoning Coherence Samples

In [ ]:
def get_reasoning_samples(events: list[CognitiveEvent], n: int = 5) -> list[dict]:
    """Get sample thinking events for manual coherence review."""
    thinking_events = [e for e in events if e.event_type == "think" and e.content]
    
    samples = []
    if thinking_events:
        samples.append(thinking_events[0])
        if len(thinking_events) > 1:
            samples.append(thinking_events[-1])
        if len(thinking_events) > 2:
            mid = len(thinking_events) // 2
            samples.append(thinking_events[mid])
        while len(samples) < min(n, len(thinking_events)):
            idx = len(samples) * len(thinking_events) // n
            if thinking_events[idx] not in samples:
                samples.append(thinking_events[idx])
            else:
                break
    
    return [
        {
            "timestamp": datetime.fromtimestamp(e.timestamp).strftime("%H:%M:%S"),
            "phase": e.phase,
            "trigger": e.metadata.get("trigger", "unspecified") if e.metadata else "unspecified",
            "content": e.content,
        }
        for e in samples
    ]

samples = get_reasoning_samples(events)
print("=== Reasoning Coherence Samples ===")
for i, sample in enumerate(samples, 1):
    print(f"--- Sample {i} ---")
    print(f"Time: {sample['timestamp']} | Phase: {sample['phase']} | Trigger: {sample['trigger']}")
    print(f"Content: {sample['content']}")
    print("-" * 60)

## 6. Summary Report

In [ ]:
def generate_summary_report(events: list[CognitiveEvent]) -> dict:
    """Generate a comprehensive summary report."""
    thinking = analyze_thinking_compliance(events)
    temptations = analyze_temptations(events)
    event_types = Counter(e.event_type for e in events)
    
    return {
        "total_events": len(events),
        "event_types": dict(event_types),
        "thinking_compliance_rate": thinking["compliance_rate"],
        "temptation_resistance_rate": temptations["resistance_rate"],
        "temptation_recognition_rate": temptations["recognition_rate"],
        "most_common_scenario": max(thinking["scenario_counts"].items(), key=lambda x: x[1])[0] if thinking["scenario_counts"] else "none",
        "most_common_temptation": max(temptations["by_type"].items(), key=lambda x: x[1])[0] if temptations["by_type"] else "none",
    }

if events:
    report = generate_summary_report(events)
    print("=" * 60)
    print("COGNITIVE ANALYSIS SUMMARY REPORT")
    print("=" * 60)
    print(f"Total cognitive events: {report['total_events']}")
    print(f"Event types: {report['event_types']}")
    print(f"Thinking compliance rate: {report['thinking_compliance_rate']:.1%}")
    print(f"Temptation recognition rate: {report['temptation_recognition_rate']:.1%}")
    print(f"Temptation resistance rate: {report['temptation_resistance_rate']:.1%}")
    print(f"Most common scenario: {report['most_common_scenario']}")
    print(f"Most common temptation: {report['most_common_temptation']}")
else:
    print("No events to analyze. Please set TRACE_ID to a valid trace ID.")

## 7. Export Report

In [ ]:
def export_report(events: list[CognitiveEvent], output_path: str = "cognitive_report.json") -> None:
    """Export full analysis report to JSON."""
    report = {
        "generated_at": datetime.now().isoformat(),
        "trace_id": TRACE_ID,
        "thinking_analysis": analyze_thinking_compliance(events),
        "temptation_analysis": analyze_temptations(events),
        "timeline": create_timeline(events),
        "reasoning_samples": get_reasoning_samples(events),
    }
    
    with open(output_path, "w") as f:
        json.dump(report, f, indent=2, default=str)
    
    print(f"Report exported to {output_path}")

# Uncomment to export:
# export_report(events, "cognitive_report.json")